In [ ]:
# HTML
import os
import re
import json
import asyncio
import nest_asyncio
from io import BytesIO
from PIL import Image, ImageOps
from playwright.async_api import async_playwright

def compute_major_px_ratio(image):
    # Compute the ratio of the most common pixel in the image
    px_count = {}
    for px in image.getdata():
        if px in px_count:
            px_count[px] += 1
        else:
            px_count[px] = 1
    max_px = max(px_count, key=px_count.get)
    return px_count[max_px] / len(image.getdata())

def process_image(image, max_size=(2560, 1440), major_px_threshold=0.98, aspect_ratio_threshold=10, filter_small=True):
    # Resize the image if it exceeds the maximum size
    if image.size[0] * image.size[1] > max_size[0] * max_size[1]:
        image.thumbnail(max_size, Image.Resampling.LANCZOS)
    
    # Convert alpha channel to white background
    if image.mode in ('RGBA', 'LA') or (image.mode == 'P' and 'transparency' in image.info):
        alpha = image.convert('RGBA').split()[-1]
        bg = Image.new("RGBA", image.size, (255, 255, 255, 255))
        bg.paste(image, mask=alpha)
        image = bg.convert("RGB")
    else:
        image = image.convert("RGB")
    major_px_ratio = compute_major_px_ratio(image)
    if major_px_ratio >= major_px_threshold:
        raise ValueError(f"Image rejected due to high major pixel ratio: {major_px_ratio:.2f}")
    byte_array = image.tobytes()
    width, height = image.size
    aspect_ratio = max(width, height) / min(width, height)

    image_from_byte_array = Image.frombytes("RGB", (width, height), byte_array)
    
    return image_from_byte_array

def extract_html_width(html):
    pattern = r'max-width:\s*(\d+)px'
    match = re.search(pattern, html)
    
    if match:
        return int(match.group(1))
    
    pattern = r'width:\s*(\d+)px'
    match = re.search(pattern, html)
    
    if match:
        return int(match.group(1))
    
    return None
def extract_html_height(html):
    pattern = r'height:\s*(\d+)px'
    match = re.search(pattern, html)
    
    if match: return int(match.group(1))
    else: return None

# Allow nested event loops
nest_asyncio.apply()

async def render_html_async(html, full_page=True, random_width=True):
    try:
        html = html.replace("initial-scale=1.0", "initial-scale=2.0")
        async with async_playwright() as p:
            browser = await p.chromium.launch()
            
            height = extract_html_height(html)
            width = extract_html_width(html)
            if width is None: width = 800
            if height is None: 
                if width is not None and width <= 800:
                    height = 600
                else:
                    height = 800

            page = await browser.new_page(viewport={"width": width, "height": height})
            
            # Collect console messages and errors
            console_errors = []
            page_errors = []
            
            # Filter out WARNING messages
            def handle_console(msg):
                if msg.type == "error" and "WARNING" not in msg.text.upper():
                    console_errors.append(f"Console {msg.type}: {msg.text}")
            
            page.on("console", handle_console)
            page.on("pageerror", lambda error: page_errors.append(f"Page Error: {str(error)}"))
            page.on("requestfailed", lambda request: page_errors.append(f"Request failed: {request.url}"))
            
            await page.set_content(html)
            await page.wait_for_timeout(5000)
            
            screenshot_bytes = await page.screenshot(full_page=full_page)
            await browser.close()
            
            # Only raise exception for real errors
            all_errors = console_errors + page_errors
            if all_errors:
                error_summary = "\n".join(all_errors)
                raise Exception(f"HTML rendering errors found:\n{error_summary}")
            
            image = Image.open(BytesIO(screenshot_bytes))
            return image
            
    except Exception as e:
        if "HTML rendering errors found" in str(e):
            raise
        raise Exception(f"Rendering failed: {str(e)}")

# Synchronous wrapper function
def render_html(html, full_page=True, random_width=True):
    return asyncio.run(render_html_async(html, full_page, random_width))

html_code = r'''<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Simple Bar Chart</title>
  <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
</head>
<body>
  <div id="plot" style="width:700px;height:500px;margin:50px auto;"></div>
  <script>
    const trace = {
      x: ['Apples', 'Bananas', 'Cherries', 'Grapes', 'Oranges'],
      y: [5, 10, 8, 6, 9],
      type: 'bar',
      marker: {
        color: ['#FF6B6B', '#FFD93D', '#6BCB77', '#4D96FF', '#FF9F1C']
      }
    };

    const layout = {
      title: 'Fruit Sales (Example)',
      xaxis: {title: 'Fruit Type'},
      yaxis: {title: 'Quantity Sold'},
      plot_bgcolor: '#f8f8f8',
      paper_bgcolor: '#fff',
      margin: {l: 60, r: 30, t: 80, b: 60}
    };

    Plotly.newPlot('plot', [trace], layout);
  </script>
</body>
</html>
'''
img = render_html(html_code)
img = process_image(img)
display(img)

In [ ]:
# Asymptote
import os
import re
import tempfile
import subprocess
from PIL import Image
from io import BytesIO

def render_asymptote(asymptote_code: str, item_id: str):
    temp_dir = tempfile.mkdtemp()
    input_asy = os.path.join(temp_dir, f"{item_id}.asy")
    output_prefix = os.path.join(temp_dir, f"{item_id}")
    output_png = output_prefix + ".png"

    setting_code = (
        "settings.prc = false;\nsettings.render = 10;\n"
        if not any(x in asymptote_code for x in ["import graph3", "import solids", "import three", "import bsp", "import contour"])
        else "settings.prc = false;\nsettings.render = 0;\n"
    )
    imports = list(re.finditer(r'^\s*import .*?;\s*$', asymptote_code, flags=re.MULTILINE))
    if imports:
        last_import = imports[-1]
        pos = last_import.end()
        asymptote_code = (
            asymptote_code[:pos] + "\n" + setting_code + "\n" + asymptote_code[pos:]
        )
    else:
        asymptote_code = setting_code + asymptote_code

    try:
        with open(input_asy, "w", encoding="utf-8") as f:
            f.write(asymptote_code)

        cmd = ["asy", "-f", "png", "-render", "10", "-o", output_prefix, input_asy]
        proc = subprocess.run(cmd, capture_output=True, text=True)

        if proc.returncode != 0:
            err = (proc.stderr or proc.stdout or "").strip()
            raise RuntimeError(f"{err}")

        with open(output_png, "rb") as f:
            image_data = f.read()
        img = Image.open(BytesIO(image_data))
        img.load()
        return img
    finally:
        for p in [input_asy, output_png]:
            try:
                if os.path.exists(p):
                    os.remove(p)
            except Exception:
                pass
        try:
            if os.path.isdir(temp_dir):
                os.rmdir(temp_dir)
        except Exception:
            pass

item_id = '''asymptote_1'''
asymptote_code = '''
size(200);

draw((-1.2,0)--(1.2,0), Arrow);
draw((0,-0.5)--(0,1.3), Arrow);

pair A = (-0.6, 0.2);
pair B = (0.8, 0.1);
pair C = (0.2, 0.9);

fill(A--B--C--cycle, lightblue + opacity(0.4));
draw(A--B--C--cycle, linewidth(1));

dot(A); dot(B); dot(C);
label("$A$", A, SW);
label("$B$", B, SE);
label("$C$", C, N);
'''
img = render_asymptote(asymptote_code, item_id)
display(img)

In [ ]:
# LilyPond
import os
import tempfile
import subprocess
import random
from io import BytesIO
from PIL import Image, ImageOps
from IPython.display import display

Image.MAX_IMAGE_PIXELS = None
random.seed(42)

def crop_background(image):
        bg_color = image.getpixel((0, 0))
        image = image.convert("RGB")
        mask = Image.new('L', image.size, 0)
        for x in range(image.width):
            for y in range(image.height):
                if image.getpixel((x, y)) == bg_color:
                    mask.putpixel((x, y), 255)
        mask = ImageOps.invert(mask)
        bbox = mask.getbbox()
        if bbox:
            buffer_size = random.randint(50, 100)
            left = max(0, bbox[0] - buffer_size)
            upper = max(0, bbox[1] - buffer_size)
            right = min(image.width, bbox[2] + buffer_size)
            lower = min(image.height, bbox[3] + buffer_size)
            return image.crop((left, upper, right, lower))
        return image

def render_lilypond(lilypond_code: str, item_id: str):
        temp_dir = tempfile.mkdtemp()
        input_ly = os.path.join(temp_dir, f"{item_id}.ly")
        output_prefix = os.path.join(temp_dir, f"{item_id}")
        output_png = output_prefix + ".png"
        try:
            with open(input_ly, "w", encoding="utf-8") as f:
                f.write(lilypond_code)

            cmd = ["lilypond", "--png", "-dno-point-and-click", "-o", output_prefix, input_ly]
            proc = subprocess.run(cmd, capture_output=True, text=True)

            if proc.returncode != 0:
                err = (proc.stderr or proc.stdout or "").strip()
                raise RuntimeError(f"{err}")

            img = Image.open(output_png)
            img.load()
            return crop_background(img)
        finally:
            for p in [input_ly, output_png]:
                try:
                    if os.path.exists(p):
                        os.remove(p)
                except Exception:
                    pass
            try:
                if os.path.isdir(temp_dir):
                    os.rmdir(temp_dir)
            except Exception:
                pass

item_id = '''lilypond_1'''
lilypond_code = '''\\version "2.22.1"

\\header {
  title = "Morning Breeze"
  composer = ""
}

melody = \\relative c'' {
  \\key g \\major
  \\time 4/4
  \\tempo "Moderato" 4 = 88

  \\dynamicUp
  g4( a) b c |
  d2 c4 b |
  a4 g fis g |
  b2. r4 |

  \\p
  e4 fis g a |
  b2 a4 g |
  fis4 e d e |
  g2. r4 |

  \\mf
  c'4 b a g |
  fis2 g4 a |
  b4( c) d b |
  a2. r4 |

  \\mp
  g4 a b a |
  g2 fis4 g |
  e4 d c b |
  g1 \\bar "|."
}

\\score {
  \\new Staff {
    \\clef treble
    \\melody
  }
  \\layout { }
  \\midi { }
}
'''
img = render_lilypond(lilypond_code, item_id)
display(img)

In [ ]:
# SVG
import cairosvg
from io import BytesIO
from PIL import Image

def render_svg(svg_string: str):
    png_data = cairosvg.svg2png(
        bytestring=svg_string.encode("utf-8")
    )
    img_buffer = BytesIO(png_data)
    return Image.open(img_buffer)

item_id = '''svg_1'''
svg_code = '''<svg width="500" height="320" xmlns="http://www.w3.org/2000/svg">
  <rect x="0" y="0" width="500" height="320" fill="white" />

  <line x1="60" y1="260" x2="460" y2="260" stroke="black" />
  <line x1="60" y1="50" x2="60" y2="260" stroke="black" />

  <rect x="90" y="180" width="30" height="80" fill="skyblue" />
  <rect x="170" y="130" width="30" height="130" fill="skyblue" />
  <rect x="250" y="100" width="30" height="160" fill="skyblue" />
  <rect x="330" y="150" width="30" height="110" fill="skyblue" />

  <polyline points="105,170 185,120 265,90 345,140"
            fill="none" stroke="orange" stroke-width="3" />

  <circle cx="105" cy="170" r="4" fill="orange" />
  <circle cx="185" cy="120" r="4" fill="orange" />
  <circle cx="265" cy="90" r="4" fill="orange" />
  <circle cx="345" cy="140" r="4" fill="orange" />

  <text x="105" y="280" text-anchor="middle" font-size="12">Exp1</text>
  <text x="185" y="280" text-anchor="middle" font-size="12">Exp2</text>
  <text x="265" y="280" text-anchor="middle" font-size="12">Exp3</text>
  <text x="345" y="280" text-anchor="middle" font-size="12">Exp4</text>

  <text x="55" y="265" text-anchor="end" font-size="12">0</text>
  <text x="55" y="215" text-anchor="end" font-size="12">100</text>
  <text x="55" y="165" text-anchor="end" font-size="12">200</text>
  <text x="55" y="115" text-anchor="end" font-size="12">300</text>
  <text x="55" y="65" text-anchor="end" font-size="12">400</text>

  <rect x="370" y="40" width="15" height="15" fill="skyblue" />
  <text x="390" y="52" font-size="12">Measured</text>
  <line x1="370" y1="70" x2="385" y2="70" stroke="orange" stroke-width="3" />
  <text x="390" y="74" font-size="12">Predicted</text>

  <text x="250" y="25" font-size="16" text-anchor="middle" font-weight="bold">
    Experimental vs Theoretical Results
  </text>
</svg>
'''
img = render_svg(svg_code)
display(img)

In [ ]:
# Mermaid
import os
import random
import tempfile
import subprocess
from pathlib import Path
from io import BytesIO
from PIL import Image
from IPython.display import display

Image.MAX_IMAGE_PIXELS = None
random.seed(42)

def render_mermaid(mermaid_code: str, mmdc_path: str, item_id: str):
    scale = random.choice([2, 3])
    temp_dir = tempfile.mkdtemp()
    input_mmd = os.path.join(temp_dir, f"{item_id}.mmd")
    output_png = os.path.join(temp_dir, f"{item_id}.png")
    try:
        with open(input_mmd, "w", encoding="utf-8") as f:
            f.write(mermaid_code)

        cmd = [mmdc_path, "-i", input_mmd, "-o", output_png, "-s", str(scale)]
        proc = subprocess.run(cmd, capture_output=True, text=True)

        if proc.returncode != 0:
            err = (proc.stderr or proc.stdout or "").strip()
            raise RuntimeError(f"{err}")

        with open(output_png, "rb") as f:
            data = f.read()
        return Image.open(BytesIO(data))
    finally:
        for path in [input_mmd, output_png]:
            if os.path.exists(path): os.remove(path)
        if os.path.isdir(temp_dir): os.rmdir(temp_dir)


mmdc_path = Path('plotting_benchmark/language_utils/env/node_modules/.bin/mmdc')

item_id = '''mermaid_2'''
mermaid_code = '''graph LR
    A[Disk1] --- B[Gateway1]
    A --- C[Gateway2]
    D[Disk2] --- B
    D --- C
    E[Disk3] --- B
    E --- C
    style A fill:#3498db,stroke:#3498db,color:#fff
    style B fill:#3498db,stroke:#3498db,color:#fff
    style C fill:#3498db,stroke:#3498db,color:#fff
    style D fill:#3498db,stroke:#3498db,color:#fff
    style E fill:#3498db,stroke:#3498db,color:#fff
    classDef label fill:none,stroke:none
    class A,B,C,D,E label
    linkStyle default stroke-width:2px,fill:none,stroke:#2c3e50'''
img = render_mermaid(mermaid_code, mmdc_path, item_id)
display(img)

In [ ]:
# Vega-Lite
import json
import pandas as pd
from pathlib import Path
import vl_convert as vlc
from io import BytesIO
from PIL import Image
import random
from wurlitzer import pipes


def is_list_of_value_dicts(data):
    if not isinstance(data, list) or len(data) == 0:
        return False
    for item in data:
        if not isinstance(item, dict) or 'value' not in item or len(item) != 1:
            return False
    return True

def load_csv_data(data_config, csv_path):
    if isinstance(data_config, dict) and "url" in data_config:
        if csv_path.exists():
            df = pd.read_csv(csv_path)
            csv_dict = df.to_dict(orient="records")
            if is_list_of_value_dicts(csv_dict):
                return {"values": [item['value'] for item in csv_dict]}
            if len(csv_dict) > 0 and 'start' in csv_dict[0] and 'stop' in csv_dict[0] and 'step' in csv_dict[0]:
                return {"sequence": csv_dict[0]}
            else:
                return {"values": csv_dict}
    return data_config

def render_vegalite(vegalite_json, csv_path):
    if "spec" in vegalite_json:
        actual_data = load_csv_data(vegalite_json["spec"].get("data", {}), csv_path)
        vegalite_json["spec"]["data"] = actual_data
    else:
        actual_data = load_csv_data(vegalite_json.get("data", {}), csv_path)
        vegalite_json["data"] = actual_data
    with pipes() as (out, err):
        png_data = vlc.vegalite_to_png(vl_spec=vegalite_json, scale=random.choice([1.5, 2, 2.5, 3]))
    log = err.read()
    if "ERROR" in log.upper():
        raise RuntimeError(f"{log}")
    img_buffer = BytesIO(png_data)
    return Image.open(img_buffer)


csv_path = Path('/mnt/tjena/yuansheng/VisCoder2-release/Code/eval/VisPlotBench/dataset/vegalite/vegalite_1.csv')

# Vega-Lite specification
vegalite_spec = '''
{
  "$schema": "https://vega.github.io/schema/vega-lite/v6.json",
  "data": {
    "values": [
      {"category": "1", "value": 4},
      {"category": "2", "value": 6},
      {"category": "3", "value": 10},
      {"category": "4", "value": 3},
      {"category": "5", "value": 7},
      {"category": "6", "value": 8}
    ]
  },
  "mark": {"type": "bar", "cornerRadiusTopLeft": 4, "cornerRadiusTopRight": 4},
  "encoding": {
    "x": {"field": "category", "type": "nominal", "title": "Category"},
    "y": {"field": "value", "type": "quantitative", "title": "Value"},
    "color": {
      "field": "value",
      "type": "quantitative",
      "scale": {"scheme": "blues"},
      "legend": {"title": "Value Intensity"}
    },
    "tooltip": [
      {"field": "category", "title": "Category"},
      {"field": "value", "title": "Value"}
    ]
  },
  "config": {
    "view": {"stroke": "transparent"},
    "axis": {"grid": false}
  }
}
'''

spec_dict = json.loads(vegalite_spec)
img = render_vegalite(spec_dict, csv_path)
display(img)

In [ ]:
# Mermaid
import os
import random
import tempfile
import subprocess
from pathlib import Path
from io import BytesIO
from PIL import Image
from IPython.display import display

Image.MAX_IMAGE_PIXELS = None
random.seed(42)

def render_mermaid(mermaid_code: str, mmdc_path: str, item_id: str):
    scale = random.choice([2, 3])
    temp_dir = tempfile.mkdtemp()
    input_mmd = os.path.join(temp_dir, f"{item_id}.mmd")
    output_png = os.path.join(temp_dir, f"{item_id}.png")
    try:
        with open(input_mmd, "w", encoding="utf-8") as f:
            f.write(mermaid_code)

        cmd = [mmdc_path, "-i", input_mmd, "-o", output_png, "-s", str(scale)]
        proc = subprocess.run(cmd, capture_output=True, text=True)

        if proc.returncode != 0:
            err = (proc.stderr or proc.stdout or "").strip()
            raise RuntimeError(f"{err}")

        with open(output_png, "rb") as f:
            data = f.read()
        return Image.open(BytesIO(data))
    finally:
        for path in [input_mmd, output_png]:
            if os.path.exists(path): os.remove(path)
        if os.path.isdir(temp_dir): os.rmdir(temp_dir)


mmdc_path = Path('plotting_benchmark/language_utils/env/node_modules/.bin/mmdc')

item_id = '''mermaid_1'''
mermaid_code = '''flowchart TD
    A[Start] --> B{Condition?}
    B -->|Yes| C[Do Step 1]
    B -->|No| D[Do Step 2]
    C --> E[End]
    D --> E
'''
img = render_mermaid(mermaid_code, mmdc_path, item_id)
display(img)

In [ ]:
import os
import tempfile
import subprocess
from shutil import rmtree
import random
import pandas as pd
import re
from PIL import Image, ImageOps
from pdf2image import convert_from_bytes
import locale

def crop_whitespace(image):
    inverted = ImageOps.invert(image.convert("RGB"))
    gray = inverted.convert("L")
    bbox = gray.getbbox()
    if bbox:
        buf = random.randint(50, 100)
        l = max(0, bbox[0] - buf)
        u = max(0, bbox[1] - buf)
        r = min(image.width, bbox[2] + buf)
        b = min(image.height, bbox[3] + buf)
        return image.crop((l, u, r, b))
    return image

def render_latex(latex_source: str, item_id: str):
    def compile_latex(compiler, tex_file, temp_dir):
        proc = subprocess.Popen(
            [compiler, "-interaction=nonstopmode", "-output-directory", temp_dir, tex_file],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
        )
        out, _ = proc.communicate()
        return proc.returncode, out

    doc_variants = []
    m = re.search(r"\\documentclass(\[.*?\])?\{(standalone|article)\}", latex_source)
    if m:
        current = m.group(2)  # standalone or article
        if current == "standalone":
            doc_variants = [ latex_source, re.sub(r"\\documentclass(\[.*?\])?\{standalone\}", r"\\documentclass{article}", latex_source)]
        else:
            doc_variants = [latex_source, re.sub(r"\\documentclass(\[.*?\])?\{article\}", r"\\documentclass{standalone}", latex_source)]
    else:
        doc_variants = [latex_source]

    temp_dir = tempfile.mkdtemp()
    try:
        last_out = None
        for variant_idx, variant_src in enumerate(doc_variants):
            tex_file = os.path.join(temp_dir, f"{item_id}_{variant_idx}.tex")
            pdf_file = os.path.join(temp_dir, f"{item_id}_{variant_idx}.pdf")
            with open(tex_file, "w", encoding="utf-8") as f:
                f.write(variant_src)

            for compiler in ["lualatex", "xelatex", "pdflatex"]:
                rc, out = compile_latex(compiler, tex_file, temp_dir)
                if rc == 0 and os.path.exists(pdf_file):
                    with open(pdf_file, "rb") as f:
                        pdf_bytes = f.read()
                    images = convert_from_bytes(pdf_bytes)
                    if images:
                        return crop_whitespace(images[0])
                else:
                    last_out = out

        if last_out is not None:
            err = last_out.decode("utf-8", errors="ignore") if isinstance(last_out, (bytes, bytearray)) else str(last_out)
            raise RuntimeError(f"LaTeX rendering failed:\n{err}")
        else:
            raise RuntimeError("LaTeX rendering failed: no compiler output captured.")
    finally:
        rmtree(temp_dir, ignore_errors=True)

os.environ["LANG"] = "C.UTF-8"
os.environ["LC_ALL"] = "C.UTF-8"
try:
    locale.setlocale(locale.LC_ALL, "C.UTF-8")
except locale.Error:
    locale.setlocale(locale.LC_ALL, "C")

item_id = '''latex_2'''
latex_code = r'''\documentclass[border=10pt]{standalone}
\usepackage{pgfplotstable}
\usepackage{pgfplots}
\pgfplotsset{compat=1.18}

% Load data directly from inline table
\pgfplotstableread[col sep=comma]{
x_value,y_value
1,2.1
2,2.9
3,3.8
}\datatable

\begin{document}
\begin{tikzpicture}
\begin{axis}[
    width=11cm,
    height=7cm,
    xlabel=Input,
    ylabel=Output,
    xmin=0, xmax=4,
    ymin=0, ymax=5,
    xtick={1,2,3,4},
    ytick={0,2,4,6},
    grid=major,
    grid style={dashed,gray!30},
    legend pos=north west,
    thick
]

% Smooth line with circular markers
\addplot[
    blue,
    mark=*,
    mark options={fill=white,scale=1.2},
    smooth,
    thick
] table[x=x_value,y=y_value] {\datatable};

\addlegendentry{Experimental data}

\end{axis}
\end{tikzpicture}
\end{document}
'''
img = render_latex(latex_code, item_id)
display(img)

In [ ]:
# Python plotly
import pandas as pd
import plotly.express as px
import plotly.io as pio

# Render as static PNG (optional: change to "browser" for interactive)
pio.renderers.default = "png"

# Create sample data directly
data = {
    "city": ["New York", "Los Angeles", "Chicago", "Houston", "Phoenix"],
    "population_mil": [8.4, 3.9, 2.7, 2.3, 1.7],
    "area_km2": [783.8, 1302, 589, 1652, 1340],
    "avg_temp_c": [12.4, 18.5, 10.3, 20.8, 23.1]
}

df = pd.DataFrame(data)

# Create scatter plot
fig = px.scatter(
    df,
    x="area_km2",
    y="population_mil",
    color="city",
    size="avg_temp_c",
    hover_data=["avg_temp_c"],
    title="City Population vs Area (Bubble Size = Avg Temperature)"
)

# Show the plot
fig.show()

In [ ]:
# Python chartify
import os
import tempfile
import pandas as pd
import chartify
from chartify import Chart
from IPython.display import Image, display
from bokeh.io.export import export_png

temp_dir = tempfile.mkdtemp()
png_file = os.path.join(temp_dir, "chartify_example.png")

data = pd.DataFrame({
    "fruit": ["Apple", "Banana", "Cherry", "Orange", "Grape"],
    "sales": [40, 55, 25, 70, 50]
})

ch = chartify.Chart(blank_labels=True, x_axis_type='categorical')

ch.plot.bar(
    data_frame=data,
    categorical_columns=["fruit"],
    numeric_column="sales"
)

ch.set_title("Fruit Sales Example")
ch.axes.set_xaxis_label("Fruit Type")
ch.axes.set_yaxis_label("Units Sold")

export_png(ch.figure, filename=png_file)

display(Image(filename=png_file))

In [ ]:
# Python bokeh
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import column
from bokeh.io import output_notebook
import numpy as np
from scipy.stats import norm
import os
import tempfile
from IPython.display import Image, display

from bokeh.io import export_svgs
import cairosvg

# Create a fixed dataset instead of reading from CSV
np.random.seed(42)
x = np.linspace(0, 10, 50)
y = 3.5 * x + 5 + np.random.normal(0, 4, size=50)  # y = 3.5x + 5 + noise
df = pd.DataFrame({"x": x, "y": y})

# Create ColumnDataSource for scatter data
source_scatter = ColumnDataSource(df)

# Fit a linear regression line (using NumPy polyfit)
slope, intercept = np.polyfit(df['x'], df['y'], 1)
reg_x = np.linspace(0, 10, 100)
reg_y = slope * reg_x + intercept
source_line = ColumnDataSource(data=dict(x=reg_x, y=reg_y))

# Create the figure
p = figure(title="Linear Relationship with Regression Line", width=800, height=600)

# Add scatter points
p.circle('x', 'y', size=8, source=source_scatter, color="#3182bd", alpha=0.7, legend_label="Observed Data")

# Add regression line
p.line('x', 'y', source=source_line, line_width=2, color="#de2d26", legend_label="Fitted Line")

# Customize the plot
p.xaxis.axis_label = "X values"
p.yaxis.axis_label = "Y values"
p.legend.location = "top_left"
p.grid.grid_line_alpha = 0.3

# Export to SVG then PNG (same as your structure)
tmpdir = tempfile.mkdtemp()
svg_file = os.path.join(tmpdir, 'python_fixed.svg')
png_file = os.path.join(tmpdir, 'python_fixed.png')
p.output_backend = 'svg'
export_svgs(p, filename=svg_file)
cairosvg.svg2png(url=svg_file, write_to=png_file)
display(Image(filename=png_file))

In [ ]:
# Python holoviews
import os
import tempfile
temp_dir = tempfile.mkdtemp()
png_file = os.path.join(temp_dir, 'python_fixed_25.png')

import pandas as pd
import holoviews as hv
import numpy as np
from IPython.display import Image, display

# Set Holoviews backend to Bokeh
hv.extension('bokeh')

# --- Fixed inline data ---
# Generate a 2D Gaussian surface (z = f(x, y))
x = np.linspace(-3, 3, 50)
y = np.linspace(-3, 3, 50)
X, Y = np.meshgrid(x, y)
Z = np.exp(-(X**2 + Y**2))  # Gaussian bump

# Convert to DataFrame for Holoviews
df_surface = pd.DataFrame({
    'x': X.ravel(),
    'y': Y.ravel(),
    'z': Z.ravel()
})

# Create contour plot
contour = hv.Contours(df_surface, kdims=['x', 'y'], vdims='z').opts(
    cmap='Viridis', colorbar=True, width=600, height=400,
    title='2D Gaussian Contour Plot', line_width=2
)

# Create scatter points (sample from Gaussian)
np.random.seed(0)
scatter_x = np.random.normal(0, 1, 80)
scatter_y = np.random.normal(0, 1, 80)
df_scatter = pd.DataFrame({'x': scatter_x, 'y': scatter_y})

scatter = hv.Scatter(df_scatter, kdims=['x'], vdims=['y']).opts(
    color='red', size=6, alpha=0.7, title='Random Samples Overlay'
)

# Overlay contour and scatter
plot = contour * scatter

# Display and save as PNG
plot
hv.save(plot, png_file, fmt='png')
display(Image(filename=png_file))


In [ ]:
# Python pyvista
import pandas as pd
import pyvista as pv
import numpy as np
import scipy as sp
from IPython.display import Image, display
import tempfile, os, signal, contextlib

@contextlib.contextmanager
def _pv_timeout(seconds=20):
    def handler(signum, frame):
        raise TimeoutError(f"PyVista operation timeout after {seconds}s")
    signal.signal(signal.SIGALRM, handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)


x = np.array([0, 1, 2, 0, 1, 2, 0, 1, 2])
y = np.array([0, 0, 0, 1, 1, 1, 2, 2, 2])
z = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0])
u = np.array([1, 0.5, -0.5, 0.8, 0.2, -0.3, 0.6, -0.4, -0.8])
v = np.array([0.5, 1, 0.8, -0.5, 0.9, 0.7, -0.6, 0.3, -0.2])
w = np.array([0.3, 0.4, 0.2, 0.6, -0.3, 0.5, 0.4, -0.5, 0.3])

plotter = pv.Plotter(off_screen=True)

points = np.column_stack((x, y, z))
vectors = np.column_stack((u, v, w))

arrows = pv.PolyData(points)
arrows['vectors'] = vectors

glyphs = arrows.glyph(orient='vectors', scale='vectors', factor=0.3)

plotter.add_mesh(glyphs, color='blue', opacity=0.5, show_scalar_bar=True)

plotter.add_text('3D Wind Field Visualization', font_size=14)
plotter.add_axes(xlabel='Longitude (X)', ylabel='Latitude (Y)', zlabel='Altitude (Z)')

tmpfile = tempfile.NamedTemporaryFile(suffix='.png', delete=False)
with _pv_timeout(20):
    plotter.screenshot(tmpfile.name)
display(Image(filename=tmpfile.name))
plotter.close()

In [ ]:
# Python matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm

x = np.linspace(0, 10, 20)
y = np.linspace(0, 8, 15)
X, Y = np.meshgrid(x, y)
Z = np.sin(X) * np.cos(Y) + 0.5 * X

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

im1 = ax1.pcolor(X, Y, Z, cmap='viridis')
ax1.set_title('Log Scale Heatmap')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
fig.colorbar(im1, ax=ax1)

im2 = ax2.pcolor(X, Y, Z, cmap='viridis')
ax2.set_title('Linear Scale Heatmap')
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
fig.colorbar(im2, ax=ax2)

fig.suptitle('Heatmap of Simulated Data')

plt.show()

In [ ]:
# Python seaborn
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

np.random.seed(42)
n = 50
data = {
    'Series A': np.cumsum(np.random.randn(n)) + 10,
    'Series B': np.cumsum(np.random.randn(n)) + 15,
    'Series C': np.cumsum(np.random.randn(n)) + 12
}
df = pd.DataFrame(data)

sns.set_palette("dark")
plt.figure(figsize=(10, 6))
sns.lineplot(data=df)
plt.title('Dark Color Palette')
plt.xlabel('Index')
plt.ylabel('Values')
plt.grid(True)


plt.show()

In [ ]:
# Python pyecharts
from pyecharts import options as opts
from pyecharts.charts import Pie
import pandas as pd
from IPython.display import Image, display
import asyncio
from playwright.async_api import async_playwright

categories = ['Electronics', 'Clothing', 'Food', 'Furniture', 'Books', 'Sports']
values = [3500, 2800, 4200, 1500, 900, 1800]

df = pd.DataFrame({
    'Category': categories,
    'Value': values
})

pie = (
    Pie()
    .add(
        "",
        [list(z) for z in zip(df['Category'], df['Value'])],
        radius=["40%", "75%"],
        center=["50%", "50%"],
        rosetype="radius",
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Market Share Distribution"),
        legend_opts=opts.LegendOpts(pos_right="20%"),
        toolbox_opts=opts.ToolboxOpts(is_show=True),
    )
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
)

pie.render_embed()
_pyecharts_objs = []
for _name in ['pie']:
    try:
        _obj = globals().get(_name, None)
        if _obj is None:
            try:
                _obj = locals()[_name]
            except Exception:
                _obj = None
        if _obj is not None and hasattr(_obj, "render_embed"):
            _pyecharts_objs.append(_obj)
    except Exception:
        pass
if not _pyecharts_objs:
    try:
        for _k, _v in list(globals().items()):
            try:
                if hasattr(_v, "render_embed"):
                    _pyecharts_objs.append(_v)
            except Exception:
                pass
    except Exception:
        pass

async def _pyecharts_display_all(_objs):
    try:
        async with async_playwright() as _p:
            _browser = await _p.chromium.launch()
            _page = await _browser.new_page()
            for _obj in _objs:
                try:
                    _html = _obj.render_embed()
                    await _page.set_content(_html)  
                    await _page.wait_for_timeout(600)
                    _png = await _page.screenshot(full_page=True)
                    display(Image(data=_png))
                except Exception as _e:
                    raise Exception(_e)
            await _browser.close()
    except Exception as _e:
        raise Exception(_e)
await _pyecharts_display_all(_pyecharts_objs)

In [ ]:
# Python altairs
import pandas as pd
import altair as alt
import numpy as np
import scipy
from IPython.display import Image, HTML, display

display(HTML('<div class="snap-anchor" data-source-id="python"></div>'))

dates = pd.date_range('2024-01-01', periods=30, freq='D')
symbols = ['AAPL'] * 30
prices = np.random.uniform(150, 180, 30)

df = pd.DataFrame({
    'date': dates,
    'symbol': symbols,
    'price': prices
})

df_filtered = df[df['symbol'] == 'AAPL']

chart = alt.Chart(df_filtered).mark_line(interpolate='step-after').encode(
    x='date:T',
    y='price:Q',
    color=alt.value('blue')
).properties(
    title='Stock Price of AAPL Over Time'
)

chart